When the German neurons are ablated, the model makes predictions on certain tokens that result in extremely high losses. Investigating these tokens, and the correct "next token" prediction, shows that many form German bigrams. The most prevalent are:

20 ('id', 'ig')
7 ('rt', 'ige')
5 (' Vert', 'rag')
5 ('he', 'ben')
4 ('ät', 'ig')
4 ('nd', 'liche')

We should run these bigrams with and without the German neurons ablated, then investigate which neurons have the most different activations at the position of the first bigram token. We'll do this using the raw logit difference.

Q: should we use single token prompts with the first token in each bigrams, or should we create prompts that end in the bigram?
A: we will create prompts the end in the bigram because models behave inconsistently for the first few tokens in each prompt, and we don't want this behaviour affecting our results.

Q: how will we create the prompts?
A: look at the word the token was used in, use GPT to generate prompts that end in this word.

20 ('id', 'ig') - Verteidigung, Verteidiger, Rechtsverteidigung
7 ('rt', 'ige') - auswärtige, sofortige, neuartigen
5 (' Vert', 'rag') - Vertragsbediensteten, vertraglichen
5 ('he', 'ben') - hervorheben, entheben
4 ('ät', 'ig') - tätig, Tätigkeit, Berufstätigen, bestätigt, gewalttätigen
4 ('nd', 'liche') - gründliche, selbstverständlichen, unmißverständlicher, ländlichen, unmissverständlichere

In [ ]:
# From GPT4

prompts = {}
words = ['Verteidigung', 'auswärtige', 'Vertragsbediensteten', 'hervorheben', 'tätig', 'gründliche']

prompts['Verteidigung'] = [
    'Ich verbringe viel Zeit mit dem Studium der Theorie und Praxis der Verteidigung',
    'In seiner Rede betonte der Minister die Notwendigkeit einer starken nationalen Verteidigung'
    'Es ist wichtig, dass wir ein geeignetes Budget für die Verteidigung',
    'Sein Fokus liegt auf der Verbesserung seiner Techniken in der Verteidigung',
    'Das Angriffsspiel ist wichtig, aber wir dürfen die Bedeutung der Verteidigung',
    'Als Anwalt hat sie viele Jahre Erfahrung in der Verteidigung',
    'Die Verteidigung',
    'In der Militärstrategie ist die beste Angriffstaktik oft eine gute Verteidigung',
    'Der Anwalt führte eine starke und überzeugende Verteidigung',
    'Die Regierung hat die Stärkung der Verteidigung',]

prompts['auswärtige'] = [
    'Meine Tätigkeit erfordert viele Reisen, daher bin ich oft auswärtige',
    'Er ist als diplomatischer Berater für alle auswärtige',
    'Der Minister für auswärtige',
    'Sie ist Expertin für auswärtige',
    'Die Behörde für auswärtige',
    'Es ist wichtig, sich über auswärtige',
    'Wir sollten uns auf die auswärtige',
    'In seiner Rolle überwacht er auswärtige',
    'Der Diplomat hat eine lange Karriere in auswärtige',
    'Die Universität bietet einen Studiengang in auswärtige']

prompts['Vertragsbediensteten'] = [
    'Nach seiner Ausbildung begann er seine Karriere als einer der Vertragsbediensteten',
    'Das Unternehmen hat eine Reihe von Vertragsbediensteten',
    'Die Rechte und Pflichten der Vertragsbediensteten',
    'Die Bezahlung der Vertragsbediensteten',
    'Wegen des hohen Arbeitsaufkommens werden zusätzliche Vertragsbediensteten',
    'Der Status der Vertragsbediensteten',
    'Die Gesundheits- und Sicherheitsvorschriften gelten auch für die Vertragsbediensteten',
    'Alle Vertragsbediensteten müssen eine Verschwiegenheitserklärung',
    'Die Firma plant, das Team der Vertragsbediensteten',
    'Die Schulung neuer Vertragsbediensteten']

prompts['hervorheben'] = [
    'In Ihrem Lebenslauf sollten Sie Ihre besonderen Fähigkeiten und Erfahrungen hervorheben',
    'Die hellen Farben im Bild sollen die Dynamik und Energie der Szene hervorheben',
    'Bei der Präsentation sollten Sie die Hauptpunkte hervorheben',
    'Die Wissenschaftler wollen die Bedeutung ihrer Forschungsergebnisse hervorheben',
    'Die Autorin nutzte Metaphern, um die Emotionen ihrer Charaktere hervorheben',
    'Mit diesem Marketingstrategieplan wollen wir die Einzigartigkeit unseres Produkts hervorheben',
    'Es ist wichtig, in der Debatte die Fakten zu hervorheben',
    'In seinem Vortrag versuchte der Redner, die Relevanz des Themas für das Publikum hervorheben',
    'Beim Design des Hauses wurde besonderer Wert darauf gelegt, die natürlichen Materialien hervorheben',
    'Im Interview konnte sie ihre umfangreichen Kenntnisse und Erfahrungen hervorheben']

prompts['tätig'] = [
    'Nach seinem Studium war er viele Jahre in der Marketingbranche tätig',
    'Sie ist als Freiwillige in einer gemeinnützigen Organisation tätig',
    'Ich bin seit über zehn Jahren als Lehrer tätig',
    'Er ist hauptsächlich in der Beratung von Start-up-Unternehmen tätig',
    'Als Journalistin war sie vor allem im politischen Bereich tätig',
    'Mein Bruder ist als Softwareentwickler tätig',
    'In ihrer Freizeit ist sie in verschiedenen sozialen Projekten tätig',
    'Nach seinem Ruhestand ist er ehrenamtlich in der Gemeinde tätig',
    'Sie ist als Autorin tätig und hat bereits mehrere Bücher veröffentlicht',
    'Als Anwalt ist er vor allem in den Bereichen Strafrecht und Zivilrecht tätig']

prompts['gründliche'] = [
    'Bevor wir mit dem Projekt fortfahren, benötigen wir eine gründliche',
    'Der Erfolg der Operation hängt von einer gründlichen',
    'Das Gesetz erfordert eine gründliche',
    'Vor dem Kauf eines Gebrauchtwagens sollte man eine gründliche',
    'Die Ermittlungen in dem Fall erfordern eine gründliche',
    'Die Studie liefert eine gründliche',
    'Das Projektteam hat eine gründliche',
    'Vor dem Abschluss des Geschäfts wird eine gründliche',
    'Die Wartung des Systems erfordert eine gründliche',
    'Die Durchführung einer gründliche']

Next we need to run through each set of prompts. For each prompt we tokenize, then traverse backwards through the model until we find the last token of our bigrams.
We concatenate any tokens after this one. This gives us our final dataset.

Next, we do a forward pass with the German neurons ablated and unablated, and save the cache. There's an existing method to do this.
We select the MLP activations at the second to last position. 
We average the MLP activations.
We compare the average difference in neuron activation at that position, and select the neurons with the largest average difference.

In [1]:
import torch
import numpy as np
from torch import einsum
from tqdm.auto import tqdm
import seaborn as sns
from transformer_lens import HookedTransformer, ActivationCache, utils
from datasets import load_dataset
from einops import einsum
import pandas as pd
from transformer_lens import utils
from rich.table import Table, Column
from rich import print as rprint
from jaxtyping import Float, Int, Bool
from torch import Tensor
import einops
import functools
from transformer_lens.hook_points import HookPoint
# import circuitsvis
from IPython.display import HTML
from plotly.express import line
import plotly.express as px
from tqdm.auto import tqdm
import json
import gc
import plotly.graph_objects as go

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from plotly.subplots import make_subplots
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from haystack_utils import load_txt_data, get_mlp_activations, line
import haystack_utils

%reload_ext autoreload
%autoreload 2

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)
model = HookedTransformer.from_pretrained("pythia-70m-v0", fold_ln=True, device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model pythia-70m-v0 into HookedTransformer


In [ ]:
for key, value in prompts.items():
    print(key)
    print(value[0])
    loss = haystack_utils.get_average_loss(prompt, model, batch_size=1, crop_context=-1, fwd_hooks=[], positionwise=False)